In [31]:
!pip install requests
!pip install BeautifulSoup
!pip install pandas
!pip install numpy

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [32]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [33]:


def get_now_playing(url, data):

    response = requests.get(url)
    # print(response.status_code)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        now_playing_section = soup.find('div', id='now-playing')  # Menemukan div dengan id 'now-showing'
        #print(now_playing_section)

        if now_playing_section:
            movies = now_playing_section.find_all('div', {'class': 'movie'})
            # print(movies)

            for movie in movies:
                title = movie.find('h4').text.strip()
                image = movie.find('img')['src']

                movie_label = movie.find('span', class_='movie-label')
                movie_type = movie_label.text.strip()
                img_tag = movie_label.find('img')
                category = img_tag['alt']
                if category == 'R13+':
                  category = 'Remaja 13 Tahun Keatas'
                elif category == 'R17+':
                  category = 'Dewasa 17 Tahun Keatas'
                elif category == 'SEUMUR HIDUP':
                  category = 'SEUMUR HIDUP'

                # print(f"Title: {title}")
                # print(f"Image: {image}")
                # print(img_alt)

                a_tag = movie.find('a')
                movie_url = a_tag['href']

                movie_response = requests.get(movie_url)
                # print(movie_response.status_code)

                if movie_response.status_code == 200:
                    movie_soup = BeautifulSoup(movie_response.content, 'html.parser')

                    movie_details = movie_soup.find('ul', class_='desc-movie').find_all('li')

                    # print(movie_detail)

                    genre_element = movie_soup.find('li', class_='movie_genre')
                    genre = genre_element.text.replace('Jenis Film', '').strip()
                    # print(genre)

                    produser_elements = movie_soup.find_all('li', class_='movie_produser')
                    # print(produser_elements)

                    for element in produser_elements:
                      span_text = element.find('span').text.strip()
                      if span_text == 'Produser':
                          produser = element.text.replace('Produser', '').strip()
                          # print(f'Produser: {produser}')
                      elif span_text == 'Sutradara':
                          sutradara = element.text.replace('Sutradara', '').strip()
                          # print(f'Sutradara: {sutradara}')

                    penulis_element = movie_soup.find('li', class_='movie_writer')
                    penulis = penulis_element.text.replace('Penulis', '').strip()
                    # print(penulis)


                    produksi_element = movie_soup.find('li', class_='movie_distributor')
                    produksi = produksi_element.text.replace('Produksi', '').strip()
                    # print(produksi)

                    casts_element = movie_soup.find('li', class_='movie_cast')
                    casts = casts_element.text.replace('Casts', '').strip()
                    # print(casts)



                    sinopsis_element = movie_soup.find('div', class_='desc-synopsis')
                    sinopsis = sinopsis_element.text.replace('SINOPSIS', '').strip()
                    # print(sinopsis_element)
                    #print(sinopsis)



                    durasi_element = movie_soup.find('div', class_='c-duration')
                    durasi = durasi_element.text.strip()
                    #print(durasi_element)
                    # print(durasi)

                    data.append([title, image, durasi, category, genre, produser, penulis, produksi, casts, sinopsis])


                else:
                    print("Failed to retrieve movie details.")

        else:
            print("Section 'Now Playing' not found.")
    else:
        print("Failed to retrieve the webpage.")


In [34]:
header = ['title', 'image', 'durasi', 'category', 'genre', 'produser', 'penulis', 'produksi', 'casts', 'sinopsis']
data = []
url = 'https://21cineplex.com/'

get_now_playing(url, data)

print("Header:", header)
print("Data:")
for row in data:
    print(row)

Header: ['title', 'image', 'durasi', 'category', 'genre', 'produser', 'penulis', 'produksi', 'casts', 'sinopsis']
Data:
['Blackpink World Tour Born Pink In Cinemas', 'https://media.21cineplex.com/webcontent/gallery/pictures/171948170811447_290x426.jpg', '92 Minutes', 'Remaja 13 Tahun Keatas', 'Music', 'Tu Sun Song', '-', 'CJ 4DPlex, ScreenX Studio', 'BLACKPINK’s JISOO, JENNIE, ROSÉ, LISA', "Tur BORN PINK BLACKPINK yang memikat dunia hadir di layar lebar, merayakan ulang tahun yang ke-8 sejak debut mereka!\n\r\nMenarik penonton sebanyak 1,8 juta dan memecahkan rekor global untuk tur konser grup perempuan, rasakan energi live performance di bioskop di seluruh dunia!\n\r\nMenampilkan set ikonik 'Hanok' yang telah memukau penggemar dan pers di seluruh dunia, film ini menunjukkan skala produksi yang belum pernah terjadi sebelumnya, menyajikan versi aransemen eksklusif dari lagu-lagu hit BLACKPINK yang unik untuk konser ini dan menghadirkan penampilan dari Gocheok Dome Seoul serta cuplikan d

In [35]:
df = pd.DataFrame(data, columns=header)

In [36]:
df.head()

,title,image,durasi,category,genre,produser,penulis,produksi,casts,sinopsis
0,Blackpink World Tour Born Pink In Cinemas,https://media.21cineplex.com/webcontent/gallery/pictures/171948170811447_290x426.jpg,92 Minutes,Remaja 13 Tahun Keatas,Music,Tu Sun Song,-,"CJ 4DPlex, ScreenX Studio","BLACKPINK’s JISOO, JENNIE, ROSÉ, LISA","Tur BORN PINK BLACKPINK yang memikat dunia hadir di layar lebar, merayakan ulang tahun yang ke-8 sejak debut mereka!\n\r\nMenarik penonton sebanyak 1,8 juta dan memecahkan rekor global untuk tur konser grup perempuan, rasakan energi live performance di bioskop di seluruh dunia!\n\r\nMenampilkan set ikonik 'Hanok' yang telah memukau penggemar dan pers di seluruh dunia, film ini menunjukkan skala produksi yang belum pernah terjadi sebelumnya, menyajikan versi aransemen eksklusif dari lagu-lagu hit BLACKPINK yang unik untuk konser ini dan menghadirkan penampilan dari Gocheok Dome Seoul serta cuplikan dari kota-kota lain di seluruh tur. \n\r\nBergabunglah bersama kami untuk semua momen menggembirakan, yang ditayangkan perdana di lebih dari 110 negara, menandai pertama kalinya bagi grup K-POP Perempuan."
1,Twisters (IMAX 2D),https://media.21cineplex.com/webcontent/gallery/pictures/171922178012807_290x426.jpg,122 Minutes,Remaja 13 Tahun Keatas,"Action, Adventure","Patrick Crowley, Frank Marshall",Mark L. Smith,Warner Bros. Pictures,"Glen Powell, Daisy Edgar-Jones, Maura Tierney, Kiernan Shipka, Katy O'Brian, David Corenswet, Anthony Ramos, Sasha Lane, Paul Scheer, Daryl McCormack, Brandon Perea",Tyler Owens (Glen Powell) dan Kate Cooper (Daisy Edgar-Jones) adalah peneliti badai dan pemburu badai yang mempertaruhkan nyawa mereka dalam upaya menguji sistem peringatan untuk bisa menanggulangi badai yang sering kali terjadi di Oklahoma.
2,Twisters,https://media.21cineplex.com/webcontent/gallery/pictures/171922168412740_290x426.jpg,122 Minutes,Remaja 13 Tahun Keatas,"Action, Adventure","Patrick Crowley, Frank Marshall",Mark L. Smith,Warner Bros. Pictures,"Glen Powell, Daisy Edgar-Jones, Maura Tierney, Kiernan Shipka, Katy O'Brian, David Corenswet, Anthony Ramos, Sasha Lane, Paul Scheer, Daryl McCormack, Brandon Perea",Tyler Owens (Glen Powell) dan Kate Cooper (Daisy Edgar-Jones) adalah peneliti badai dan pemburu badai yang mempertaruhkan nyawa mereka dalam upaya menguji sistem peringatan untuk bisa menanggulangi badai yang sering kali terjadi di Oklahoma.
3,The Exorcism,https://media.21cineplex.com/webcontent/gallery/pictures/171818351464876_290x426.jpg,95 Minutes,Dewasa 17 Tahun Keatas,"Horror, Thriller","Bill Block, Ben Fast, Kevin Williamson","M.A. Fortin, Joshua John Miller",Miramax,"Russell Crowe, Ryan Simpkins, Sam Worthington, Chloe Bailey, Adam Goldberg, Adrian Pasdar, David Hyde Pierce, Tracey Bonner, Marcenae Lynette, Joshua John Miller",Anthony Miller (Russell Crowe) aktor senior yang sedang bermasalah mulai menunjukkan prilaku aneh saat terlibat dalam produksi film horor. Lee Miller (Ryan Simpkins) putrinya bingung apakah sang ayah kembali ke kecanduan masa lalu atau ada sesuatu yang lebih jahat yang sedang terjadi.
4,Kalki 2898 AD,https://media.21cineplex.com/webcontent/gallery/pictures/171921603696425_290x426.jpg,180 Minutes,SEMUA UMUR,"Action, Sci-Fi","Aswini Dutt, Swapna Dutt, Priyanka Dutt",Kotagiri Venkatesawara Rao,Vyjayanthi Movies,"Prabhas, Deepikia Padukone, Amitabh Bachchan, Disha Patani, Kamal Haasan","Pada tahun 2898 M, di kota Kasi yang sepi, sekarang menjadi satu-satunya kota yang ada di dunia, diperintah oleh elit totaliter, dipimpin oleh Supreme Yaskin (Kamal Hasaan), dengan megastruktur piramida terbalik melayang di atas kota yang dikenal sebagai Complex. Berlatar belakang mitologi India dan masyarakat distopia, kisah ini mengisahkan perjalanan selama ribuan tahun, mulai dari peristiwa Mahabharata pada tahun 3102 SM, permulaan Kali Yuga, hingga tahun 2898 M, munculnya Avatar modern dewa Hindu Wisnu, dikatakan turun ke Bumi untuk melindungi dunia dari kekuatan jahat."


In [38]:
df.to_csv('output-webscraping-cinema21.csv', index=False)

from google.colab import files
files.download('output-webscraping-cinema21.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>